In [1]:
import pandas as pd
import numpy as np
import pyspark

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
myRange = spark.range(1000).toDF("number")

In [9]:
myRange.show(5)

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows



In [12]:
divisBy2 = myRange.where("number % 2 = 0")

In [13]:
divisBy2.show(5)

+------+
|number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
+------+
only showing top 5 rows



In [14]:
divisBy2.count()

500

### http://localhost:4040
for your spark UI

In [16]:
flightData2015 = spark\
.read\
.option("inferSchema", "true")\
.option("header", "true")\
.csv("2015-summary.csv")

In [17]:
flightData2015.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [18]:
flightData2015.sort("count").explain()

== Physical Plan ==
*(1) Sort [count#58 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(count#58 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#153]
   +- FileScan csv [DEST_COUNTRY_NAME#56,ORIGIN_COUNTRY_NAME#57,count#58] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/brian.clements/Documents/Certifications_study_work/databricks/Da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [19]:
spark.conf.set("spark.sql.shuffle.partitions","5")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [20]:
spark.conf.set("spark.sql.shuffle.partitions","4")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

In [21]:
spark.conf.set("spark.sql.shuffle.partitions","16")
flightData2015.sort("count").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

## Create sql temp view

In [23]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [24]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

In [25]:
dataFrameWay = flightData2015.groupBy("DEST_COUNTRY_NAME").count()

In [27]:
sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#56, 16), ENSURE_REQUIREMENTS, [id=#197]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#56] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/brian.clements/Documents/Certifications_study_work/databricks/Da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#56, 16), ENSURE_REQUIREMENTS, [id=#216]
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[partial_count(1)])
      +- FileScan csv [DEST_COUNTRY_NAME#56] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/brian.clements/Documents/Certifications_study_work/da

In [30]:
# sqlWay.show(100)

In [31]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

[Row(max(count)=370002)]

In [33]:
from pyspark.sql.functions import max
flightData2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [34]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

In [35]:
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [37]:
from pyspark.sql.functions import desc

flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.show(5)

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+
only showing top 5 rows



In [42]:
flightData2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#234L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#56,destination_total#234L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[sum(cast(count#58 as bigint))])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#56, 16), ENSURE_REQUIREMENTS, [id=#479]
      +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#56], functions=[partial_sum(cast(count#58 as bigint))])
         +- FileScan csv [DEST_COUNTRY_NAME#56,count#58] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/brian.clements/Documents/Certifications_study_work/databricks/Da..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>


